**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [1]:
from datetime import datetime
from pathlib import Path
import sys

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data

# The Neighbor

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/3/

Sarah and the investigator were very impressed with your data skills, as you were able to figure out the phone number of the contractor. They called up the cleaning contractor straight away and asked about the rug.

“Oh, yeah, I did some special projects for them a few years ago. I remember that rug unfortunately. I managed to clean one section, which revealed a giant spider that startled me whenever I tried to work on it.

“I already had a fear of **spiders** before this, but this spider was so realistic that I had a hard time making any more progress. I kept expecting the cleaners would call for the rug, but they never did. I felt so bad about it, I couldn’t face them, and of course they never gave me another project.

“At last I couldn’t deal with the rug taking up my whole bathtub, so I gave it to this guy **who lived in my neighborhood**. He said that he was naturally intuitive because he was a **Cancer born in the year of the Rabbit**, so maybe he was able to clean it.

“I don’t remember his name. Last time I saw him, he was leaving the subway and carrying a bag from Noah’s. I swore I saw a **spider on his hat**.”

Can you find the phone number of the person that the contractor gave the rug to?


In [2]:
data = get_data(data_path=Path('./../data/5784/'))
df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

In [30]:
YEARS_OF_THE_RABBIT = [2035, 2023, 2011, 1999, 1987, 1975, 1963, 1951, 1939, 1927]
CANCER_SIGN = {"start": {"month": 6, "day": 22}, "end": {"month": 7, "day": 22}}
CONTRACTOR_NUMBER = "332-274-4185"
CONTRACTOR_CITY = (
    df_customers
    .select(pl.col("citystatezip", "address", "phone"))
    .filter(pl.col("phone") == CONTRACTOR_NUMBER)
).get_column("citystatezip").item()


df_cancers_year_rabbit = (
    df_customers
    .select(
        pl.col(["customerid", "name", "address", "citystatezip", "phone", "birthdate"]),       
    )
    .filter(
        (pl.col("birthdate").dt.year().is_in(YEARS_OF_THE_RABBIT)) & 
        (((pl.col("birthdate").dt.month() == CANCER_SIGN["start"]["month"]) & (pl.col("birthdate").dt.day() >= CANCER_SIGN["start"]["day"])) | ((pl.col("birthdate").dt.month() == CANCER_SIGN["end"]["month"]) & (pl.col("birthdate").dt.day() <= CANCER_SIGN["end"]["day"]))) &
        (pl.col("citystatezip") == CONTRACTOR_CITY)
    )
)

print("Day 3:", df_cancers_year_rabbit.select("phone").item())